In [1]:
%%time
import math
import torch

from botorch.test_functions import SixHumpCamel
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.utils.transforms import standardize, normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import (
    ExpectedImprovement,
    ProbabilityOfImprovement,
    qMaxValueEntropy,
)
import matplotlib.pyplot as plt
from botorch.optim import optimize_acqf
from botorch.acquisition import qNoisyExpectedImprovement, qProbabilityOfImprovement
from time import time
import cProfile, io, pstats
from pstats import SortKey
import scipy

SMOKE_TEST = True

torch.manual_seed(123456)
torch.set_default_dtype(torch.double)

bounds = torch.tensor(SixHumpCamel._bounds).T
bounds_norm = torch.tensor([[0.0, 0.0], [1.0, 1.0]])
train_X = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(5, 2)
train_Y = SixHumpCamel(negate=True)(train_X).unsqueeze(-1)

train_X = normalize(train_X, bounds=bounds)
train_Y = standardize(train_Y + 0.05 * torch.randn_like(train_Y))

model = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll, max_attempts=10)

from botorch.acquisition.max_value_entropy_search import qLowerBoundMaxValueEntropy

candidate_set_size = 1000 if not SMOKE_TEST else 5
candidate_set = torch.rand(
    candidate_set_size, bounds_norm.size(1), device=bounds.device, dtype=bounds.dtype
)
qGIBBON = qLowerBoundMaxValueEntropy(model, candidate_set)
scipy.__version__

CPU times: user 4.03 s, sys: 3.21 s, total: 7.24 s
Wall time: 21.1 s


'1.7.3'

In [2]:
%%time

# prep different acqusition functions
candidate_set = torch.rand(
    10000, bounds.size(1), device=bounds.device, dtype=bounds.dtype
)
acqs = {}
# acqs["aa"] = qLowerBoundMaxValueEntropy(model, candidate_set)
# acqs["MES"] = qMaxValueEntropy(model, candidate_set)
acqs["EI"] = qNoisyExpectedImprovement(model, train_X)
# acqs["PI"] = qProbabilityOfImprovement(model, best_f=train_Y.max())

# prep grid to evaluate acq functions
n = 100 if not SMOKE_TEST else 2
xv, yv = torch.meshgrid([torch.linspace(0, 1, n), torch.linspace(0, 1, n)])
test_x = torch.stack([xv.reshape(n * n, 1), yv.reshape(n * n, 1)], -1)

# eval and maximise acq functions

with cProfile.Profile() as pr:
    for acq in acqs.values():
        acq(test_x).detach().reshape(n, n)
        optimize_acqf(
            acq_function=acq,
            bounds=bounds_norm,
            q=3,
            num_restarts=5,
            raw_samples=100,
            sequential=True,
        )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats()

scipy.__version__

         133342 function calls (122730 primitive calls) in 0.172 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.170    0.170 /opt/anaconda3/lib/python3.9/site-packages/botorch/optim/optimize.py:415(optimize_acqf)
        1    0.000    0.000    0.170    0.170 /opt/anaconda3/lib/python3.9/site-packages/botorch/optim/optimize.py:547(_optimize_acqf)
        1    0.000    0.000    0.170    0.170 /opt/anaconda3/lib/python3.9/site-packages/botorch/optim/optimize.py:216(_optimize_acqf_sequential_q)
        3    0.000    0.000    0.170    0.057 /opt/anaconda3/lib/python3.9/site-packages/botorch/optim/optimize.py:258(_optimize_acqf_batch)
        3    0.000    0.000    0.160    0.053 /opt/anaconda3/lib/python3.9/site-packages/botorch/optim/optimize.py:294(_optimize_batch_candidates)
        3    0.000    0.000    0.159    0.053 /opt/anaconda3/lib/python3.9/site-packages/botorch/generation/gen.py:43

/opt/anaconda3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]



       89    0.000    0.000    0.000    0.000 {method 'clamp_min' of 'torch._C._TensorBase' objects}
       89    0.000    0.000    0.000    0.000 /opt/anaconda3/lib/python3.9/site-packages/botorch/posteriors/gpytorch.py:41(__init__)
      267    0.000    0.000    0.000    0.000 /Users/saitcakmak/linear_operator/linear_operator/operators/_linear_operator.py:1892(ndimension)
        3    0.000    0.000    0.000    0.000 /opt/anaconda3/lib/python3.9/site-packages/botorch/sampling/qmc.py:60(draw)
     1068    0.000    0.000    0.000    0.000 /opt/anaconda3/lib/python3.9/site-packages/gpytorch/settings.py:93(on)
       82    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(array_equal)
      623    0.000    0.000    0.000    0.000 {built-in method _pickle.dumps}
       89    0.000    0.000    0.000    0.000 /opt/anaconda3/lib/python3.9/site-packages/gpytorch/means/constant_mean.py:85(constant)
       89    0.000    0.000    0.000    0.000 {built-in method torch.split_with

'1.7.3'